In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [3]:
gm_eligibility = client.to_pandas("""

WITH events AS (
  SELECT *
  FROM oso.int_superchain_events_by_project
  WHERE    
    "time" BETWEEN DATE('2024-10-01') AND DATE('2025-02-28')
    AND event_type IN ('CONTRACT_INVOCATION', 'CONTRACT_INTERNAL_INVOCATION')
    AND project_id = 'klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs='
),

builder_metrics AS (
  SELECT
    project_id,
    TRUE as applied_to_round,
    COUNT(DISTINCT transaction_hash) AS transaction_count,
    COUNT(DISTINCT from_artifact_id) AS active_addresses_count,
    COUNT(DISTINCT DATE_TRUNC('DAY', events.time)) AS active_days,
    SUM(events.gas_fee) AS gas_fees
  FROM events
  GROUP BY project_id
),

project_eligibility AS (
  SELECT
    project_id,
    (
      CAST(transaction_count >= 1000 AS INTEGER) +
      CAST(active_days >= 10 AS INTEGER) +
      CAST(active_addresses_count >= 420 AS INTEGER) +
      CAST(gas_fees >= 0.0 AS INTEGER)
      >= 3
    ) AS meets_all_criteria
  FROM builder_metrics
),

artifacts AS (
  SELECT DISTINCT
    project_id,
    'DEFILLAMA_ADAPTER' AS artifact_type
  FROM oso.artifacts_by_project_v1
  WHERE artifact_source = 'DEFILLAMA'

  UNION ALL

  SELECT DISTINCT
    p2p.external_project_id AS project_id,
    'BUNDLE_BEAR' AS artifact_type
  FROM oso.int_projects_to_projects AS p2p
  JOIN oso.artifacts_by_project_v1 AS abp
    ON p2p.artifact_id = abp.artifact_id
  WHERE
    p2p.ossd_project_name IN (
      SELECT DISTINCT ossd_name
      FROM oso.int_4337_address_labels
    )
    AND p2p.external_project_source = 'OP_ATLAS'
    AND abp.artifact_source = 'GITHUB'
),

artifact_flags AS (
  SELECT
    project_id,
    MAX(CASE WHEN artifact_type = 'DEFILLAMA_ADAPTER' THEN TRUE ELSE FALSE END) AS has_defillama_adapter,
    MAX(CASE WHEN artifact_type = 'BUNDLE_BEAR' THEN TRUE ELSE FALSE END) AS has_bundle_bear
  FROM artifacts
  GROUP BY 1
)

SELECT
  builder_metrics.project_id,
  COALESCE(builder_metrics.transaction_count, 0) AS transaction_count,
  COALESCE(builder_metrics.gas_fees, 0.0) AS gas_fees,
  COALESCE(builder_metrics.active_addresses_count, 0) AS active_addresses_count,
  COALESCE(builder_metrics.active_days, 0) AS active_days,
  builder_metrics.applied_to_round,
  project_eligibility.meets_all_criteria,
  (
    builder_metrics.applied_to_round
    AND project_eligibility.meets_all_criteria
  ) AS is_eligible,
  COALESCE(artifact_flags.has_defillama_adapter, FALSE)
    AS has_defillama_adapter, 
  COALESCE(artifact_flags.has_bundle_bear, FALSE)
    AS has_bundle_bear
FROM builder_metrics
JOIN project_eligibility
  ON builder_metrics.project_id = project_eligibility.project_id
LEFT JOIN artifact_flags
  ON builder_metrics.project_id = artifact_flags.project_id

""")
gm_eligibility.head()

,project_id,transaction_count,gas_fees,active_addresses_count,active_days,applied_to_round,meets_all_criteria,is_eligible,has_defillama_adapter,has_bundle_bear
0,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,2299605,2.018321,158928,39,True,True,True,False,False


In [4]:
gm_eligibility.iloc[0]

project_id                klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=
transaction_count                                              2299605
gas_fees                                                      2.018321
active_addresses_count                                          158928
active_days                                                         39
applied_to_round                                                  True
meets_all_criteria                                                True
is_eligible                                                       True
has_defillama_adapter                                            False
has_bundle_bear                                                  False
Name: 0, dtype: object

In [5]:
gm_metrics = client.to_pandas("""

WITH base_events AS (
  SELECT
    e.project_id,
    DATE_TRUNC('DAY', e.time) AS bucket_day,
    DATE_TRUNC('MONTH', e.time) AS bucket_month,
    e.event_type,
    e.event_source AS chain,
    e.from_artifact_id,
    e.to_artifact_id,
    e.gas_fee,
    e.transaction_hash,
    COALESCE(users.is_farcaster_user, false) AS is_farcaster_user
  FROM oso.int_superchain_events_by_project AS e
  LEFT OUTER JOIN oso.int_superchain_onchain_user_labels AS users
  ON e.from_artifact_id = users.artifact_id
  WHERE
    /* Currently no 4337-specific logic as these events are 
    already captured in the CONTRACT_INTERNAL_INVOCATION
    events and would need to be de-duped. */ 
    e.event_type IN (
      'CONTRACT_INVOCATION',
      'CONTRACT_INTERNAL_INVOCATION'
    )
    AND e.time >= DATE('2025-01-01') AND e.time < DATE('2025-03-01')
    AND e.project_id = 'klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs='
),

-- Amortized transaction counts
amortized_transaction_count AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'amortized_contract_invocations_monthly' AS metric_name,
    COUNT(DISTINCT transaction_hash) AS amount
  FROM base_events
  WHERE event_type = 'CONTRACT_INTERNAL_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Transaction counts
transaction_count AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'contract_invocations_monthly' AS metric_name,
    COUNT(DISTINCT transaction_hash) AS amount
  FROM base_events
  WHERE event_type = 'CONTRACT_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Gas fees
transaction_gas_fee AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'gas_fees_monthly' AS metric_name,
    SUM(gas_fee) AS amount
  FROM base_events AS e
  WHERE event_type = 'CONTRACT_INVOCATION'
  GROUP BY 1, 2, 3
),

-- Active Farcaster users
monthly_active_farcaster_users AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'active_farcaster_users_monthly' AS metric_name,
    COUNT(DISTINCT from_artifact_id) AS amount
  FROM base_events
  WHERE is_farcaster_user = true
  GROUP BY 1, 2, 3
),

-- Active addresses
monthly_active_addresses AS (
  SELECT
    project_id,
    chain,
    bucket_month AS sample_date,
    'active_addresses_monthly' AS metric_name,
    COUNT(DISTINCT from_artifact_id) AS amount
  FROM base_events
  GROUP BY 1, 2, 3
),

union_all_metrics AS (
  SELECT *
  FROM amortized_transaction_count
  UNION ALL
  SELECT *
  FROM transaction_count
  UNION ALL
  SELECT *
  FROM transaction_gas_fee
  UNION ALL
  SELECT *
  FROM monthly_active_farcaster_users
  UNION ALL
  SELECT *
  FROM monthly_active_addresses
)

SELECT
  project_id AS project_id,
  chain AS chain,
  sample_date AS sample_date,
  metric_name AS metric_name,
  amount AS amount
FROM union_all_metrics

""")

gm_metrics['project_name'] = '0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c150e3f72d3dc7ef22dd71a'
gm_metrics['display_name'] = 'GM'
gm_metrics['sample_date'] = gm_metrics['sample_date'].apply(lambda x: x.split(' ')[0])
gm_metrics['measurement_period'] = gm_metrics['sample_date'].apply(lambda x: "Jan 2025" if x == "2025-01-01" else "Feb 2025")

gm_metrics = gm_metrics[[
    'project_id','display_name','project_name','chain',
    'metric_name','sample_date','measurement_period','amount'    
]]

In [6]:
gm_metrics

,project_id,display_name,project_name,chain,metric_name,sample_date,measurement_period,amount
0,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,contract_invocations_monthly,2025-02-01,Feb 2025,1830400.0
1,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,contract_invocations_monthly,2025-01-01,Jan 2025,531606.0
2,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,gas_fees_monthly,2025-02-01,Feb 2025,1.439588
3,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,gas_fees_monthly,2025-01-01,Jan 2025,0.692692
4,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,active_addresses_monthly,2025-01-01,Jan 2025,126386.0
5,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,active_addresses_monthly,2025-02-01,Feb 2025,129805.0
6,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,active_farcaster_users_monthly,2025-02-01,Feb 2025,21948.0
7,klIcJBxU6pe2YYzCfGsvu/mm08cTveDuOFmn7v9JwRs=,GM,0x2c1d2ae68e4a718a04a41f1a9fde1ed0648602e08c15...,INK,active_farcaster_users_monthly,2025-01-01,Jan 2025,21147.0
